# 下载数据集
    https://www.kaggle.com/competitions/california-house-prices/leaderboard

In [16]:
%matplotlib inline
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.utils  import data 

# 读取数据

In [2]:
train_path="dataset//train.csv"
test_path="dataset//test.csv"
train_data=pd.read_csv(train_path)
test_data=pd.read_csv(test_path)
train_data.shape,test_data.shape

((1460, 81), (1459, 80))

In [3]:
train_data.iloc[0:4,[0,1,2,3,-3,-2,-2,-1]]

,Id,MSSubClass,MSZoning,LotFrontage,SaleType,SaleCondition,SaleCondition,SalePrice
0,1,60,RL,65.0,WD,Normal,Normal,208500
1,2,20,RL,80.0,WD,Normal,Normal,181500
2,3,60,RL,68.0,WD,Normal,Normal,223500
3,4,70,RL,60.0,WD,Abnorml,Abnorml,140000


ID这个特征，不具有预测的信息，所以将其删除

In [4]:
all_data=pd.concat((train_data.iloc[:,1:-1],test_data.iloc[:,1:]))  #去除了价格
all_data.shape

(2919, 79)

# 数据预处理
* 数据标准化（归一化）
* 数值型特征工程
* 离散型特征工程
* 张量数据转化

数据标准化：u代表均值，o代表标准差

<img src="pic\1.jpg" />

In [5]:
'''
如果数据不是一个object，那么该数据就是数值型
'''
numeric_features=all_data.dtypes[all_data.dtypes!='object'].index  #数值类型的数据的属性名
no_numeric_features=all_data.dtypes[all_data.dtypes=='object'].index  #非数值类型
print("非数值型",no_numeric_features)
print("数值型",numeric_features)
all_data[numeric_features] = all_data[numeric_features].apply(        #将数值型数据归一化
        lambda x: (x - x.mean()) / (x.std()))
#在标准化数据之后，所有均值消失，因此我们可以将缺失值设置为0
all_data[numeric_features] =all_data[numeric_features].fillna(0)

非数值型 Index(['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
       'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature',
       'SaleType', 'SaleCondition'],
      dtype='object')
数值型 Index(['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
       'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr

处理离散值，我们用one-hot编码替换他们，方法与前面将多类别的标签转化为向量的方式相同

In [8]:
#“Dummy_na=True”将“na”（缺失值）视为有效的特征值，并为其创建指示符特征
all_data=pd.get_dummies(all_data,dummy_na=True)
all_data.shape

(2919, 331)

从pandas格式中提取numpy格式，并将其转化为张量表示用于训练.

In [13]:
n_train=train_data.shape[0]
train_f=torch.tensor(all_data[:n_train].values,dtype=torch.float32)  #训练数据
test_f=torch.tensor(all_data[n_train:].values,dtype=torch.float32)   #测试数据
train_label=torch.tensor(train_data.SalePrice.values.reshape(-1,1),dtype=torch.float32)  #标签

train_f.shape,test_f.shape,train_label.shape

(torch.Size([1460, 331]), torch.Size([1459, 331]), torch.Size([1460, 1]))

In [20]:
'''数据迭代器'''
def data_iter(data_arrays,batch_size):
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset,batch_size,shuffle=True)


# 模型训练

## 参数定义

In [18]:
batch_size=64
num_epochs=3
weight_decay=0.04
dropout1=0.2
dropout2=0.4
lr=0.1

## 定义模型

In [ ]:
#一个简单的线性模型作为Baseline
def model(in_features):
    net=torch.nn.Sequential(
        torch.nn.Linear(in_features,1)
    )
    return net

房价就像股票价格一样，我们关心的是相对数量，而不是绝对数量。我们更关心相对误差（y-y_hat/yy_hat/y），而不是绝对误差（y-y_hat）。例如，在预测上海的一套房子偏差了40万，然而真正价值是60万，那么模型就会显得很糟糕。另一方面，如果在汤臣一品的预测出现10万偏差，然而真实价格是1000万，这可能是一个不错的预测。

解决这个问题的一种方法是用价格预测的对数来衡量差异。预测价格的对数与真实标签价格的对数之间出现以下均方根误差（用来替换的loss来判定一个模型的好坏）

<img src="pic\2.jpg" />

In [22]:
def log_rmse(train_f,train_label):
    # 为了在取对数时进一步稳定该值，将小于1的值设置为1，将数据控制在1~正无穷
    clip_pred=torch.clamp(net(train_f),1,float("inf"))  #预测的数值
    rmse=torch.sqrt(torch.log(clip_pred,train_label))
    return rmse.item()

与前面不同的是，我们的训练函数借助Adam优化器（后续会讲到）。 Adam优化器的主要吸引力在于它对初始学习率不那么敏感。

In [ ]:
optimizer=torch.optim.Adam(net.parameters(),lr=lr,weight_decay=weight_decay)  #这里利用权重衰退，控制模型参数

In [21]:
train_iter=data_iter((train_f,train_label),batch_size)  #生成数据迭代器
test_iter=data_iter(test_f,batch_size)

### 训练

In [ ]:
'''定义预测准确率函数'''
def acc(y_hat,y):
    '''
    :param y_hat: 接收二维张量，例如 torch.tensor([[1], [0]...])
    :param y: 接收二维张量，例如 torch.tensor([[0.1, 0.2, 0.7], [0.8, 0.1, 0.1]...]) 三分类问题
    :return:
    '''
    y_hat=y_hat.argmax(axis=1)
    cmp=y_hat.type(y.dtype)==y  #数据类型是否相同
    return float(cmp.type(y.dtype).sum())
    
class Accumulator():
    ''' 对评估的正确数量和总数进行累加 '''
    def __init__(self, n):
        self.data = [0.0] * n

    def add(self, *args):
        self.data = [a + float(b) for a, b in zip(self.data, args)]

    def reset(self):
        self.data = [0.0] * len(self.data)

    def __getitem__(self, item):
        return self.data[item]

'''自定义每个批次训练函数'''
def train_epoch_cha3(net,train_iter,loss,optimizer):
    #判断是不是pytorch得model，如果是，就打开训练模式，pytorch得训练模式默认开启梯度更新
    if isinstance(net,torch.nn.Module):
        net.train()
    #创建样本累加器【累加每批次的损失值、样本预测正确的个数、样本总数】
    metric = Accumulator(3)  
    for x,y in train_iter:
        #前向传播获取预测结果
        y_hat=net(x)
        #计算损失
        l=loss(y_hat,y) 
        #判断是pytorch自带得方法还是我们手写得方法（根据不同得方法有不同得处理方式）
        if isinstance(optimizer,torch.optim.Optimizer):
            #梯度清零
            optimizer.zero_grad()
            #损失之求和，反向传播（pytorch自动进行了损失值计算）
            l.backward()
            #更新梯度
            optimizer.step()
            #累加个参数
            metric.add(
                float(l)*len(y),  #损失值总数
                acc(y_hat,y),     #计算预测正确得总数
                y.size().numel()  #样本总数
            )
    #返回平均损失值，预测正确得概率
    return metric[0]/metric[2],metric[1]/metric[2]    

'''模型测试'''
def test_cha3(net,test_iter):
    if isinstance(net,torch.nn.Module):
        net.eval()  #将模型设置为评估模式
    metric=Accumulator(2)
    for x,y in test_iter:
        metric.add(
            acc(net(x),y),  #计算准确个数
            y.numel()  #测试样本总数
        )
    #返回模型得准确率
    print(f"test_acc={metric[0]/metric[1]:.2f}%")
    return metric[0]/metric[1]

'''正式训练'''
def train_cha3(num_epochs,net,train_iter,test_iter,loss,optimizer):
    loss_list=[]
    train_acc=[]
    test_acc=[]
    for epoch in range(num_epochs):
        #计算训练数据的平均损失值和正确率
        train_metrics=train_epoch_cha3(net,train_iter,loss,optimizer)
        loss_list.append(train_metrics[0])  #保存loss
        train_acc.append(train_metrics[1])   #保存准确率
        #计算验证集的准确率
        test_metrics=test_cha3(net,test_iter)
        test_acc.append(test_metrics)
        print(f"epoch{epoch+1}:loss={train_metrics[0]},train_acc={train_metrics[1]*100:.2f}%,test_acc={test_metrics:.2f}%")
                
    return loss_list,train_acc,test_acc



In [ ]:
loss=torch.nn.MSELoss()  #均方损失
in_features=train_f.shape[1]  #特征数量
net=model(in_features)
train_cha3(num_epochs,net,train_iter,test_iter,loss,optimizer)